# Introduction & Dependencies

https://opencorporates.com/

In [1]:
%load_ext jupyter_ai

In [2]:
%load_ext dotenv

In [41]:
%dotenv 

In [44]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Allow named parameters (python variables) in SQL cells
%config SqlMagic.named_parameters=True

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


/Users/me/projects/new-york-real-estate/.venv/lib/python3.10/site-packages/sql/traits.py:20: FutureWarning: named_parameters: boolean values are now deprecated. Value True will be treated as "enabled". 
Please use a valid option: "warn", "enabled", or "disabled". 
For more information, see the docs: https://jupysql.ploomber.io/en/latest/api/configuration.html#named-parameters
  warnings.warn(


# Test on one state: gather context for language model

In [12]:
%%capture path_structure
import seedir as sd
path = '~/data/opencorporates'
sd.seedir(path, style='lines', depthlimit=2, exclude_folders=['.git', '.ipynb_checkpoints'])

In [13]:
print(path_structure.stdout)

opencorporates/
├─us_mi/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_mn/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_nh/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_al/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_ky/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_ak/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_names.csv.gz
├─us_mt/
│ ├─companies.csv.gz
│ ├─officers.csv.gz
│ ├─additional_identifiers.csv.gz
│ ├─non_reg_addresses.csv.gz
│ └─alternative_nam

In [29]:
%%capture ny_companies
!gzcat ~/data/opencorporates/us_ny/companies.csv.gz | head -n 5

In [30]:
print(ny_companies.stdout)

company_number,jurisdiction_code,name,normalised_name,company_type,nonprofit,current_status,incorporation_date,dissolution_date,branch,business_number,current_alternative_legal_name,current_alternative_legal_name_language,home_jurisdiction_text,native_company_number,previous_names,retrieved_at,registry_url,restricted_for_marketing,inactive,accounts_next_due,accounts_reference_date,accounts_last_made_up_date,annual_return_next_due,annual_return_last_made_up_date,has_been_liquidated,has_insolvency_history,has_charges,number_of_employees,registered_address.street_address,registered_address.locality,registered_address.region,registered_address.postal_code,registered_address.country,registered_address.in_full,home_jurisdiction_code,home_jurisdiction_company_number,industry_code_uids,latest_accounts_date,latest_accounts_cash,latest_accounts_assets,latest_accounts_liabilities
1000000,us_ny,TAYLOR RAND LTD.,taylor rand limited,DOMESTIC BUSINESS CORPORATION,false,Inactive   Dissolution By Procl

In [31]:
%%capture ny_officers
!gzcat ~/data/opencorporates/us_ny/officers.csv.gz | head -n 5

In [78]:
ny_officers.stdout

'id,company_number,jurisdiction_code,name,title,first_name,last_name,position,start_date,person_number,person_uid,end_date,current_status,occupation,nationality,country_of_residence,partial_date_of_birth,type,address.in_full,address.street_address,address.locality,address.region,address.postal_code,address.country,retrieved_at,source_url\r\n72342016,3332175,us_ny,WAJAD AZIMI,,,,agent,,,,,,,,,,,"363 S BROADWAY, YONKERS, NEW YORK, 10705",,,,,,2016-04-22 16:58:01 UTC,\r\n224809571,3785280,us_ny,BENEDETTA AMADI,,,,chief executive officer,,,,,,,,,,,"BENEDETTA AMADI, BROOKLYN, NY, 11211",BENEDETTA AMADI,BROOKLYN,NY,11211,,2024-05-13 11:37:21 UTC,\r\n224940888,2067108,us_ny,RICHARD P. SZMYR,,,,chief executive officer,,,,,,,,,,,"RICHARD P. SZMYR, SCOTIA, NY, 12302",RICHARD P. SZMYR,SCOTIA,NY,12302,,2024-05-06 11:39:24 UTC,\r\n225116861,2116899,us_ny,RON BUKSHPAN,,,,chief executive officer,,,,,,,,,,,"RON BUKSHPAN, CUMMING, GA, 30041",RON BUKSHPAN,CUMMING,GA,30041,,2024-05-06 11:39:24 UTC,\r\n'

In [32]:
%%capture ny_additional_identifiers
!gzcat ~/data/opencorporates/us_ny/additional_identifiers.csv.gz | head -n 5

In [33]:
%%capture ny_non_reg_addresses
!gzcat ~/data/opencorporates/us_ny/non_reg_addresses.csv.gz | head -n 5

In [34]:
%%capture ny_alternative_names
!gzcat ~/data/opencorporates/us_ny/alternative_names.csv.gz | head -n 5

## Design the prompt that can be re-used for every file we want to load and visualize

In [97]:
parent_prompt = f"""
Please take the following context for the directory structure: 

{path_structure.stdout}

Then consider only the `ny` folder, with the following files and headers:

`companies.csv.gz`: 

{ny_companies.stdout}

`officers.csv.gz`:

{ny_officers.stdout}

`additional_identifiers.csv.gz`:

{ny_additional_identifiers.stdout}

`alternative_names.csv.gz`:

{ny_alternative_names.stdout}

Proceed step-by-step to copy the resulting database into a parquet file compressed with ZSTD compression, using the duckdb dialect of SQL, for future use in a dbt model, in the `~/data/opencorporates` directory, sharded in the same way (by state), only for the ny state.

Remember to also proceed step-by-step as an elite site reliability/devops/L20 principal warez engineer at google, returning as few tokens as possible, to debug this SQL code. Give the complete corrected code!

Please always remember to prefix the output with `%%sql` for the JupySQL cell magic :)
"""

In [39]:
!ls -lh ~/data/opencorporates/us_ny

total 7582728
-rwxr--r--  1 me  staff   159K May 22 11:37 additional_identifiers.csv.gz*
-rwxr--r--  1 me  staff   482K May 22 11:37 alternative_names.csv.gz*
-rwxr--r--@ 1 me  staff   2.9G May 22 12:55 companies.csv*
-rwxr--r--@ 1 me  staff   438M May 22 11:38 companies.csv.gz*
-rwxr--r--  1 me  staff    18M May 22 11:38 non_reg_addresses.csv.gz*
-rwxr--r--  1 me  staff   235M May 22 11:38 officers.csv.gz*


# Execute the prompt once for every file

In [98]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file in `~/data/opencorporates/us_ny`:

```
-rwxr--r--  1 me  staff   159K May 22 11:37 additional_identifiers.csv.gz*
```

In [99]:
%%sql
COPY
(
  SELECT
    company_number,
    jurisdiction_code,
    uid,
    identifier_system_code
  FROM read_csv_auto('~/data/opencorporates/us_ny/additional_identifiers.csv.gz', header=True, sep=',')
)
TO '~/data/opencorporates/additional_identifiers.parquet'
(FORMAT 'PARQUET', CODEC 'ZSTD');

,Success


In [100]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file in `~/data/opencorporates/us_ny`:

```
-rwxr--r--  1 me  staff   482K May 22 11:37 alternative_names.csv.gz*
```

In [101]:
%%sql
CREATE TABLE ny_alternative_names AS 
SELECT * 
FROM read_csv_auto('~/data/opencorporates/us_ny/alternative_names.csv.gz', header=True, sep=',');

COPY (
    SELECT *
    FROM ny_alternative_names
) TO '~/data/opencorporates/us_ny/alternative_names.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [115]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file in `~/data/opencorporates/us_ny`:

```
-rwxr--r--@ 1 me  staff   2.9G May 22 12:55 companies.csv.gz
```

Remember that if a column name contains a period, then you need to use double quotes around the entire column name. Do NOT use the `read_csv_auto` function and instead specify the type of every column in the `read_csv` options and parameters.

Do NOT use the read_csv_auto function, and specify all the columns in the read_csv parameter. Only give the final code and do not load any duckdb extensions nor create tables; one copy command suffices :)

Instead of the `read_csv_auto` function, remember to use the `read_csv` function and specify all types correctly with the `columns` parameter :) 
Instead of the `read_csv_auto` function, remember to use the `read_csv` function and specify all types correctly with the `columns` parameter :) 
Instead of the `read_csv_auto` function, remember to use the `read_csv` function and specify all types correctly with the `columns` parameter :) 
Instead of the `read_csv_auto` function, remember to use the `read_csv` function and specify all types correctly with the `columns` parameter :) 

In [119]:
%%sql
COPY (
  SELECT
    company_number,
    jurisdiction_code,
    name,
    normalised_name,
    company_type,
    nonprofit,
    current_status,
    incorporation_date,
    dissolution_date,
    branch,
    business_number,
    current_alternative_legal_name,
    current_alternative_legal_name_language,
    home_jurisdiction_text,
    native_company_number,
    previous_names,
    retrieved_at,
    registry_url,
    restricted_for_marketing,
    inactive,
    accounts_next_due,
    accounts_reference_date,
    accounts_last_made_up_date,
    annual_return_next_due,
    annual_return_last_made_up_date,
    has_been_liquidated,
    has_insolvency_history,
    has_charges,
    number_of_employees,
    "registered_address.street_address",
    "registered_address.locality",
    "registered_address.region",
    "registered_address.postal_code",
    "registered_address.country",
    "registered_address.in_full",
    home_jurisdiction_code,
    home_jurisdiction_company_number,
    industry_code_uids,
    latest_accounts_date,
    latest_accounts_cash,
    latest_accounts_assets,
    latest_accounts_liabilities
  FROM read_csv(
    '~/data/opencorporates/us_ny/companies.csv.gz',
    columns={
      'company_number': 'VARCHAR',
      'jurisdiction_code': 'VARCHAR',
      'name': 'VARCHAR',
      'normalised_name': 'VARCHAR',
      'company_type': 'VARCHAR',
      'nonprofit': 'BOOLEAN',
      'current_status': 'VARCHAR',
      'incorporation_date': 'DATE',
      'dissolution_date': 'DATE',
      'branch': 'VARCHAR',
      'business_number': 'VARCHAR',
      'current_alternative_legal_name': 'VARCHAR',
      'current_alternative_legal_name_language': 'VARCHAR',
      'home_jurisdiction_text': 'VARCHAR',
      'native_company_number': 'VARCHAR',
      'previous_names': 'VARCHAR',
      'retrieved_at': 'TIMESTAMP',
      'registry_url': 'VARCHAR',
      'restricted_for_marketing': 'BOOLEAN',
      'inactive': 'BOOLEAN',
      'accounts_next_due': 'DATE',
      'accounts_reference_date': 'DATE',
      'accounts_last_made_up_date': 'DATE',
      'annual_return_next_due': 'DATE', 
      'annual_return_last_made_up_date': 'DATE',
      'has_been_liquidated': 'BOOLEAN',
      'has_insolvency_history': 'BOOLEAN',
      'has_charges': 'BOOLEAN',
      'number_of_employees': 'INTEGER',
      'registered_address.street_address': 'VARCHAR',
      'registered_address.locality': 'VARCHAR',
      'registered_address.region': 'VARCHAR',
      'registered_address.postal_code': 'VARCHAR',
      'registered_address.country': 'VARCHAR',
      'registered_address.in_full': 'VARCHAR',
      'home_jurisdiction_code': 'VARCHAR',
      'home_jurisdiction_company_number': 'VARCHAR',
      'industry_code_uids': 'VARCHAR',
      'latest_accounts_date': 'DATE',
      'latest_accounts_cash': 'DOUBLE',
      'latest_accounts_assets': 'DOUBLE',
      'latest_accounts_liabilities': 'DOUBLE'
    }
  )
)
TO '~/data/opencorporates/us_ny/companies.parquet'
(FORMAT PARQUET, CODEC ZSTD, ROW_GROUP_SIZE 100000);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [120]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file in `~/data/opencorporates/us_ny`:

```
-rwxr--r--  1 me  staff    18M May 22 11:38 non_reg_addresses.csv.gz*
```

Make sure to only use the correct columns that actually appear in the header you have been provided. And make sure to specify the read_csv options manually rather than use the auto one. (do NOT use the `read_csv_auto` function!).

In [71]:
%%sql
COPY (
SELECT * 
FROM read_csv('~/data/opencorporates/us_ny/non_reg_addresses.csv.gz',
    auto_detect=False,
  delim=',',
    null_padding=true,
  header=True,
  columns={
    'company_number': 'VARCHAR',
    'jurisdiction_code': 'VARCHAR',
    'value': 'VARCHAR',
    'service_address': 'VARCHAR',
    'address.in_full': 'VARCHAR',
    'address.street_address': 'VARCHAR',
    'address.locality': 'VARCHAR',
    'address.region': 'VARCHAR',
    'address.postal_code': 'VARCHAR',
    'address.country': 'VARCHAR',
    'type': 'VARCHAR',
    'start_date': 'DATE',
    'end_date': 'DATE'
  }
)
    ) TO '~/data/opencorporates/us_ny/non_reg_addresses.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

,Success


In [82]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{parent_prompt}

Only do this for this file in `~/data/opencorporates/us_ny`:

```
-rwxr--r--  1 me  staff   235M May 22 11:38 officers.csv.gz*
```

Remember that if a column name contains a period, then you need to use double quotes around the entire column name. Do NOT use the `read_csv_auto` function and instead specify the type of every column in the `read_csv` options and parameters.

In [86]:
%%sql
CREATE TABLE officers AS 
SELECT 
  id::INTEGER,
  company_number,
  jurisdiction_code,
  name,
  title,
  first_name,  
  last_name,
  position,
  start_date::DATE,
  person_number,
  person_uid,
  end_date::DATE,
  current_status,
  occupation,
  nationality,
  country_of_residence,
  partial_date_of_birth::DATE,
  type,
  "address.in_full",
  "address.street_address",
  "address.locality",
  "address.region",
  "address.postal_code",
  "address.country",
  retrieved_at::TIMESTAMP,
  source_url
FROM read_csv_auto('~/data/opencorporates/us_ny/officers.csv.gz', delim=',', header=True, columns={
  'id': 'INT',
  'company_number': 'VARCHAR',
  'jurisdiction_code': 'VARCHAR',
  'name': 'VARCHAR',
  'title': 'VARCHAR',
  'first_name': 'VARCHAR',
  'last_name': 'VARCHAR',
  'position': 'VARCHAR',
  'start_date': 'DATE',
  'person_number': 'VARCHAR',
  'person_uid': 'VARCHAR',
  'end_date': 'DATE',
  'current_status': 'VARCHAR',
  'occupation': 'VARCHAR',
  'nationality': 'VARCHAR',
  'country_of_residence': 'VARCHAR',
  'partial_date_of_birth': 'DATE',
  'type': 'VARCHAR',
  'address.in_full': 'VARCHAR',
  'address.street_address': 'VARCHAR', 
  'address.locality': 'VARCHAR',
  'address.region': 'VARCHAR',
  'address.postal_code': 'VARCHAR',
  'address.country': 'VARCHAR',
  'retrieved_at': 'TIMESTAMP',
  'source_url': 'VARCHAR'
});

COPY officers TO '~/data/opencorporates/us_ny/officers.parquet' (FORMAT 'parquet', CODEC 'ZSTD');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


# Prototype visualizations and network analyses

In [89]:
%%capture file_list
!ls ~/data/opencorporates/us_ny/*.parquet

In [121]:
%%capture officers_parquet
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/officers.parquet' LIMIT 10;"

In [122]:
%%capture non_reg_addresses_parquet
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/non_reg_addresses.parquet' LIMIT 10;"

In [123]:
%%capture companies_parquet
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/companies.parquet' LIMIT 10;"

In [124]:
%%capture alternative_names_parquet
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/alternative_names.parquet' LIMIT 10;"

In [126]:
%%capture docs

!curl {https://uwdata.github.io/mosaic/jupyter/}

In [145]:
%%capture example

!curl {https://pastebin.com/raw/XijHp75S}

In [159]:
prompt = f"""
Take the following as context: 

```
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/officers.parquet' LIMIT 10;"

{officers_parquet.stdout}
```

```
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/non_reg_addresses.parquet' LIMIT 10;"

{non_reg_addresses_parquet.stdout}
```

```
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/companies.parquet' LIMIT 10;"

{companies_parquet.stdout}
```

```
!duckdb -markdown -c "SELECT * FROM '~/data/opencorporates/us_ny/alternative_names.parquet' LIMIT 10;"

{alternative_names_parquet.stdout}
```
"""

In [160]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{prompt}

Use the Jupyter widget and the above example to visualize what is reasonable in the companies.parquet file and header. debug it. proceed step-by-step as an elite site reliability/devops/L20 principal warez engineer at google, returning as few tokens as possible, to debug this visualization code using the reference. give the complete corrected code!

In [162]:
import ipywidgets as widgets
import duckdb

parquet_file = "~/data/opencorporates/us_ny/companies.parquet"
conn = duckdb.connect()

# Read the parquet file into a DuckDB table
conn.execute(f"CREATE TABLE companies AS SELECT * FROM read_parquet('{parquet_file}')")

# Get the table header
header = conn.execute("SELECT * FROM companies LIMIT 0").description
header = [col[0] for col in header]

# Get the list of company jurisdictions
jurisdictions = conn.execute("SELECT DISTINCT jurisdiction_code FROM companies ORDER BY jurisdiction_code").fetchall()
jurisdictions = [j[0] for j in jurisdictions]

# Create widgets
jurisdiction_dropdown = widgets.Dropdown(options=jurisdictions, description='Jurisdiction:', value=jurisdictions[0])
limit_text = widgets.IntText(value=10, description='Limit:', min=1, max=1000)

# Define the query function
def query_companies(jurisdiction, limit):
    query = f"SELECT * FROM companies WHERE jurisdiction_code = '{jurisdiction}' LIMIT {limit}"
    return conn.execute(query).fetchdf()

# Create the output widget
output = widgets.Output()

# Define the on_click function
def on_click(b):
    with output:
        output.clear_output()
        df = query_companies(jurisdiction_dropdown.value, limit_text.value)
        print(df.to_markdown(index=False))

# Create the button widget
button = widgets.Button(description="Query")
button.on_click(on_click)

# Display the widgets
display(jurisdiction_dropdown, limit_text, button, output)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Dropdown(description='Jurisdiction:', options=('us_ny',), value='us_ny')

IntText(value=10, description='Limit:')

Button(description='Query', style=ButtonStyle())

Output()

In [163]:
%%ai anthropic-chat:claude-3-opus-20240229 --format code

{prompt}

Use the Jupyter widget and the above example to visualize what is reasonable in the companies.parquet file and header. debug it. proceed step-by-step as an elite site reliability/devops/L20 principal warez engineer at google, returning as few tokens as possible, to debug this visualization code using the reference. give the complete corrected code!

Here is the documentation for the Jupyter widget we will be using for visualization: from https://uwdata.github.io/mosaic/jupyter/

{example_raw}

ValueError: Invalid format specifier